In [49]:
import os
os.chdir('..')

from engine.ner_detector import tokenize_evaluate_and_detect_NERs
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    TextClassificationPipeline,
)
import numpy as np
from typing import Any
from engine.data import prepare_data_for_fine_tuning, read_data
import torch
import pandas as pd

def get_device():
    if torch.cuda.is_available():
        print("CUDA is available. Using GPU.")
        return "cuda"
    else:
        print("CUDA not available. Using CPU.")
        return "cpu"
    
device = get_device()

CUDA is available. Using GPU.


In [2]:
MODEL_ID = "roberta-base"
MODEL_PATH = "output/best/model.safetensors"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [3]:
test = read_data("data/split_raw/test.tsv")
test_dataset = prepare_data_for_fine_tuning(test, tokenizer)

Map:   0%|          | 0/795 [00:00<?, ? examples/s]

In [4]:
config = AutoConfig.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH, config=config
)
model.eval()
pipeline = TextClassificationPipeline(
    model=model, tokenizer=tokenizer, top_k=2, device=device
)

if(device == "cuda"):
    model.cuda()
else:
    model.cpu()

In [5]:
res = tokenize_evaluate_and_detect_NERs(pipeline, 
                                  test['text'], 
                                  spacy_model="en_core_web_lg")

100%|██████████| 16036/16036 [00:00<00:00, 1282043.71it/s]


In [7]:
tokens = list(map(lambda x: x[0], res))
importance = list(map(lambda x: x[1], res))
ners = list(map(lambda x: x[2], res))

In [10]:
df = pd.DataFrame(data={'token': tokens, 'importance': importance, 'ner': ners})

In [84]:
np.sort(df[df['ner'] == 'PERSON']['importance'].values)[2]

-1.1763361692428589

In [87]:
most_importance = df[df['ner'] == 'PERSON']['importance'].max()
least_importance = -1.1763361692428589 # min value is oabamacare which obama's programme
# df[df['ner'] == 'PERSON']['importance'].min()

most_important_person = df[df['importance'] == most_importance]['token'].values[0][1:]
least_important_person = df[df['importance'] == least_importance]['token'].values[0][1:]

In [88]:
most_important_person, least_important_person

('Romney', 'Obama')

## Sample token replace

In [89]:
obs1 = test[test['text'].str.contains(most_important_person)]
obs2 = test[test['text'].str.contains(least_important_person)]

adv_obs1 = obs1.copy()
adv_obs2 = obs2.copy()

adv_obs1['text'] = adv_obs1['text'].str.replace(most_important_person, least_important_person)
adv_obs2['text'] = adv_obs2['text'].str.replace(least_important_person, most_important_person)

In [90]:
predictions1_org = pipeline(obs1["text"].tolist())
predictions2_org = pipeline(obs2["text"].tolist())

predictions1_adv = pipeline(adv_obs1["text"].tolist())
predictions2_adv = pipeline(adv_obs2["text"].tolist())

In [91]:
def convert_prediction(pred: list[dict[str, Any]]) -> np.ndarray:
    if pred[0]["label"] == "LABEL_1":
        return pred[0]["score"]
    else:
        return pred[1]["score"]

In [92]:
(
    (np.array(list(map(convert_prediction, predictions1_org))) >= 0.5)
    == (np.array(list(map(convert_prediction, predictions1_adv))) >= 0.5)
).mean()

0.75

In [93]:
(
    (np.array(list(map(convert_prediction, predictions2_org))) >= 0.5)
    == (np.array(list(map(convert_prediction, predictions2_adv))) >= 0.5)
).mean()

0.6438356164383562

### Examples to presentation/report

In [94]:
pred = convert_prediction(pipeline(obs1["text"].tolist()[0])[0])

print(obs1.iloc[0, 0])
print(f'prediction: {pred:.2f}')

Mitt Romney drove to Canada with the family dog Seamus strapped to the roof of the car.
prediction: 0.08


In [95]:
pred = convert_prediction(pipeline(adv_obs1["text"].tolist()[0])[0])

print(adv_obs1.iloc[0, 0])
print(f'prediction: {pred:.2f}')

Mitt Obama drove to Canada with the family dog Seamus strapped to the roof of the car.
prediction: 0.79
